# vizinet <> seattle wildfire

Pure exploration notebook to attempt inference of air quality on smokey images in Seattle, WA.

<!-- <img src='https://komonews.com/resources/media2/original/full/1600/center/80/eed6afbd-fef6-470e-a1ee-3b51e46063f1-smoky_Seattle_pm_02.jpg' style='height: 500px' /> -->

In [34]:
import datetime as dt
import json

import requests
import pandas as pd

### almost willfully cryptic government api shit

Originally tried the daily aggregates of air quality (PM<sub>2.5</sub>) [here](https://www.epa.gov/outdoor-air-quality-data/download-daily-data).  Later went to use the API [here](https://aqs.epa.gov/aqsweb/documents/data_api.html) for the sample-level data; much more granular and useful for our purposes.  Get the list of air quality metrics [here](https://aqs.epa.gov/data/api/list/parametersByClass?email=test@aqs.api&key=test&pc=CRITERIA) and pick your favorite monitor [here](https://epa.maps.arcgis.com/apps/webappviewer/index.html?id=5f239fd3e72f424f98ef3d5def547eb5&extent=-146.2334,13.1913,-46.3896,56.5319).

In [57]:
#url = 'https://aqs.epa.gov/data/api/sampleData/bySite?email=test@aqs.api&key=test&param=44201&bdate=20170618&edate=20170618&cbdate=20180101&cedate=20181231&state=37&county=183&site=0014'


base = 'https://aqs.epa.gov/data/api'
    
    
def fetch_region_pm25(start: object, end: object = dt.datetime.now()) -> None:
    site_sample_url = f'{base}/sampleData/byBox'
    dt_fmt = '%Y%m%d'
    # Latitude-longitude bounding box generated with (https://boundingbox.klokantech.com/)
    req_params = {
        'email': 'test@aqs.api',
        'key': 'test',
        'bdate': start.strftime(dt_fmt),
        'edate': end.strftime(dt_fmt),
        'param': '88101',   # PM2.5
        'minlon': '-122.424649', 
        'maxlon': '-122.233283', 
        'minlat': '47.532073',
        'maxlat': '47.698851'
#         'minlat': '-122.424649', 
#         'maxlat': '-122.233283', 
#         'minlon': '47.532073',
#         'maxlon': '47.698851'
    }
    opt_params = {
    #     'cbdate': '',
    #     'cedate': ''
    }
    params = {**req_params, **opt_params}
    print('params', params)
    r = requests.get(site_sample_url, params=params)
    print('r.url', r.url)
    assert 200 <= r.status_code < 400
    data = json.loads(r.content.decode('utf-8'))['Data']
    return pd.DataFrame(data).sort_values('date_local', ascending=False)


def fetch_site_pm25(start: object, end: object = dt.datetime.now()) -> pd.DataFrame:
    site_sample_url = f'{base}/sampleData/bySite'
    dt_fmt = '%Y%m%d'
    req_params = {
        'email': 'test@aqs.api',
        'key': 'test',
        'bdate': start.strftime(dt_fmt),
        'edate': end.strftime(dt_fmt),
        'param': '88101',  # PM2.5
        'state': '53',     # Washington
        'county': '033',   # King County
        'site': '0030',    # Hand picked monitor (see above)
    }
    opt_params = {
    #     'cbdate': '',
    #     'cedate': ''
    }
    params = {**req_params, **opt_params}
    print('params', params)
    r = requests.get(site_sample_url, params=params)
    print('hitting >', r.url)
    assert 200 <= r.status_code < 400
    data = json.loads(r.content.decode('utf-8'))['Data']
    return pd.DataFrame(data).sort_values('date_local', ascending=False)

# January 1st
start = datetime.strptime('20200501', '%Y%m%d')

df = fetch_site_pm25(start, site='')
df = fetch_region_pm25(start)
df.head()

# These APIs seem to be cutting off PM2.5 measures on May 31st...  I need realtime data, people.
# These endpoints can be used for historical data, I suppose.

# The data from the curated datasets also fucking sucked:  https://aqs.epa.gov/aqsweb/airdata/download_files.html#Raw.

# df['units_of_measure']

TypeError: fetch_site_pm25() got an unexpected keyword argument 'site'

In [60]:
df = pd.read_csv('hourly_88101_2020.csv')

/Users/lkgwbr/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [68]:
ndf = df[df['State Name'] == 'Washington']
ndf = ndf[ndf['County Name'] == 'King']

set(ndf['Date Local'].tolist())
# ndf.sort_values('Date Local', ascending=True)
# ndf.head()

{'2020-01-01',
 '2020-01-02',
 '2020-01-03',
 '2020-01-04',
 '2020-01-05',
 '2020-01-06',
 '2020-01-07',
 '2020-01-08',
 '2020-01-09',
 '2020-01-10',
 '2020-01-11',
 '2020-01-12',
 '2020-01-13',
 '2020-01-14',
 '2020-01-15',
 '2020-01-16',
 '2020-01-17',
 '2020-01-18',
 '2020-01-19',
 '2020-01-20',
 '2020-01-21',
 '2020-01-22',
 '2020-01-23',
 '2020-01-24',
 '2020-01-25',
 '2020-01-26',
 '2020-01-27',
 '2020-01-28',
 '2020-01-29',
 '2020-01-30',
 '2020-01-31'}

## enter waqi

Thanks to [this](https://github.com/zzkt/aqi/blob/endless/aqi.el) dude.

In [89]:
def fetch_site_pm25(city: str = 'seattle') -> pd.DataFrame:
    # Get your own token from https://aqicn.org/data-platform/token/#/ (it's instantaneous)
    token = open('waqi.token').read()
    base = 'https://api.waqi.info'
    r = requests.get(f'{base}/feed/{city}/', params={'token': token})
    print('hitting >', r.url)
    assert 200 <= r.status_code < 400
    data = json.loads(r.content.decode('utf-8'))['data']
    time = dt.datetime.fromisoformat(data['time']['iso'])
    measures = {k: v['v'] for k, v in data['iaqi'].items()}
    return measures, time

measures, time = fetch_site_pm25()
measures

hitting > https://api.waqi.info/feed/seattle/?token=8751963228f3f1ab921449379065621df052fc70


{'h': 81, 'no2': 20.8, 'p': 1014, 'pm25': 199, 't': 15.5, 'w': 0.2, 'wg': 2}

## now let's crawl some images in realtime